In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score

In [2]:
top = 100
split_pair_path = "../data/split_dev/split_pair_dev_k_{0}.txt".format(top)
train_data_path = "../data/split_dev/train_set.txt"
dev_data_path = "../data/split_dev/dev_set_v1_4.txt"


train_data = pd.read_csv(train_data_path, sep='\t', header=None)
train_label = np.array(train_data.iloc[:,0]).astype('int')
dev_data = pd.read_csv(dev_data_path, sep='\t', header=None)
dev_label = np.array(dev_data.iloc[:,0]).astype('int')
split_data = pd.read_csv(split_pair_path, sep='\t')

In [3]:
split_yes_no_list = np.array(split_data.iloc[:,2])
print("positive:negative = {0}:{1}".format(sum(split_yes_no_list==1), sum(split_yes_no_list==0)))

positive:negative = 177881:3104819


In [4]:
%%time

total_single_split_num = int(len(split_yes_no_list)/top)
counter = 0
index_list = []
for i in range(0, len(split_yes_no_list), top):
    single_split = split_yes_no_list[i:(i+top)]
    if 1 in single_split:
        #new_train_data = new_train_data.append(split_data.iloc[i:(i+top),:])
        index_list.append(i)
        counter += 1
print("recall: {0}/{1}={2:.4f}".format(counter, total_single_split_num, (counter/total_single_split_num)))

recall: 16574/32827=0.5049
CPU times: user 93.4 ms, sys: 137 µs, total: 93.5 ms
Wall time: 193 ms


In [ ]:
%%time

new_train_data = pd.DataFrame()
for i in range(0, len(split_yes_no_list), top):
    single_split = split_yes_no_list[i:(i+top)]
    if 1 in single_split:
        new_train_data = new_train_data.append(split_data.iloc[i:(i+top),:])

In [ ]:
ratio = 1/ (sum(new_train_data.iloc[:,2]==1)/sum(new_train_data.iloc[:,2]==0))
print("positive:negative = {0}:{1}".format(sum(new_train_data.iloc[:,2]==1), sum(new_train_data.iloc[:,2]==0)))
print("ratio: {0}".format(1/ratio))
print("ratio: {0}".format(ratio))

In [ ]:
split_pair_path = "../data/split_dev/new_split_pair_dev_k_{0}.txt".format(top)

new_train_data.to_csv(split_pair_path, sep='\t', index=False)
#new_train_data = pd.read_csv(split_pair_path, sep='\t')
new_train_data = new_train_data.reset_index(drop=True)

In [167]:
# from probability file, get top k candidates
probability_file_path = "../data/split_dev/dev_results_k_100.csv"

probability_file = pd.read_csv(probability_file_path, sep='\t',header=None).iloc[:,1]


In [168]:
%%time
total_len = len(new_train_data)
unique_len = int(total_len/top)
index_set = set(range(unique_len))
true_label = np.array(new_train_data.true_label).reshape(unique_len, top)[:,1]
candidate_label_mat = np.array(new_train_data.candidate_label).reshape(unique_len, top)
probability_mat = np.array(probability_file).reshape(unique_len, top)

CPU times: user 29.4 ms, sys: 0 ns, total: 29.4 ms
Wall time: 8.44 ms


In [196]:
def getIndexedList(target_mat, index_list):
    return target_mat[np.arange(len(target_mat)), index_list]

def getMaxProbabilityList(probability_mat, candidate_label_mat):
    max_probability_index = np.argmax(probability_mat, axis=1)
    max_probability_list = getIndexedList(probability_mat, max_probability_index)
    max_label_list = getIndexedList(candidate_label_mat, max_probability_index)
    return max_probability_list, max_label_list

def computeAccuracy(predicted, true_label):
    accuracy = accuracy_score(true_label, predicted)
    print("accuracy: {0}".format(accuracy))

In [238]:
%%time
#Iterative Deepening
max_top = 100
top_step_size = 20
new_candidate_list = np.zeros((unique_len,1)).astype('int')
index_list = np.array(list(index_set))
probability_threshold = 0.5
for mini_top in range(top_step_size, max_top, top_step_size):
    max_probability_list, max_label_list = \
            getMaxProbabilityList( \
                probability_mat[index_list, :mini_top],
                candidate_label_mat[index_list, :mini_top])

#     mini_probability_mat = probability_mat[index_list, :mini_top]
#     mini_label_mat = candidate_label_mat[index_list, :mini_top]
#     max_probability_index = np.argmax(mini_probability_mat)
#     max_probability_list = getIndexedList(mini_probability_mat, max_probability_index)
#     max_label_list = getIndexedList(mini_label_mat, max_probability_index)
   
    filted_probablity_index = (max_probability_list > probability_threshold)
    new_index_list = index_list[filted_probablity_index]
    temp_max_candidate_list = max_label_list[filted_probablity_index]
    temp_max_candidate_list = temp_max_candidate_list.reshape((len(temp_max_candidate_list),1))
    new_candidate_list[new_index_list] = temp_max_candidate_list

    #new_candidate_list[index_list][filted_probablity_index] = temp_max_candidate_list
    #new_candidate_list[index_list] = max_label_list.reshape((len(max_label_list),1))
    #print(np.sum(new_candidate_list == 0))
    print(len(new_index_list))
    computeAccuracy(temp_max_candidate_list, true_label[new_index_list])
    index_list = index_list[np.logical_not(filted_probablity_index)]
    

max_probability_list, max_label_list = \
        getMaxProbabilityList(
            probability_mat[index_list, :max_top],
            candidate_label_mat[index_list, :max_top])
print(len(index_list))
new_candidate_list[index_list] = max_label_list.reshape((len(max_label_list), 1))
computeAccuracy(new_candidate_list, true_label)

10116
accuracy: 0.842131277184658
728
accuracy: 0.49313186813186816
448
accuracy: 0.44642857142857145
261
accuracy: 0.4482758620689655
5021
accuracy: 0.6116206105949077
CPU times: user 19 ms, sys: 0 ns, total: 19 ms
Wall time: 18.4 ms


accuracy: 0.6103535658259925


In [68]:
true_label

array([2129,  262,  420, ..., 5098, 4935, 7549])

11553